In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [2]:
train_df = pd.read_csv('../data/preprocessed_train.csv')

In [13]:
train_df['accuracy_group'].value_counts(normalize=True)

3    0.500000
0    0.239062
1    0.136292
2    0.124647
Name: accuracy_group, dtype: float64

In [12]:
train_df.columns.tolist()

['0_group_count',
 '1_group_count',
 '2_group_count',
 '3_group_count',
 'accumulated_accuracy',
 'accumulated_accuracy_group',
 'accumulated_actions',
 'accumulated_correct_attempts',
 'accumulated_sessions',
 'accumulated_uncorrect_attempts',
 'accuracy_group',
 'assess_duration_mean',
 'assess_duration_sum',
 'changed_type_Activity_count',
 'changed_type_Assessment_count',
 'changed_type_Clip_count',
 'changed_type_Game_count',
 'current_title_count',
 'current_title_total_time',
 'current_world_activity_time',
 'current_world_count',
 'current_world_game_time',
 'current_world_total_time',
 'day',
 'days_since_installation',
 'hour',
 'installation_id',
 'last_accuracy_Bird Measurer (Assessment)',
 'last_accuracy_Cart Balancer (Assessment)',
 'last_accuracy_Cauldron Filler (Assessment)',
 'last_accuracy_Chest Sorter (Assessment)',
 'last_accuracy_Mushroom Sorter (Assessment)',
 'sec_since_installation',
 'title',
 'title_12 Monkeys_count',
 'title_Air Show_count',
 'title_All Star 

In [8]:
train_df['hour'].max()

23

In [55]:
test['timestamp'] = pd.to_datetime(test['timestamp'])

In [96]:
(test['timestamp'][1000] - test['timestamp'][2]).days

12

array(['NONE', 'MAGMAPEAK', 'CRYSTALCAVES', 'TREETOPCITY'], dtype=object)

In [100]:
train_df['world'].unique()

array([3, 1, 0])

In [103]:
test['title'].nunique()

44

In [105]:
(test['title'] + test['type']).nunique()

44

In [85]:
test['timestamp']

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10 16:50:24.910000+00:00,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10 16:50:55.503000+00:00,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10 16:51:51.805000+00:00,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10 16:53:12.825000+00:00,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10 16:54:12.115000+00:00,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES
...,...,...,...,...,...,...,...,...,...,...,...
1156409,c74f40cd,46ff9d3ad2be09f2,2019-09-28 21:20:40.918000+00:00,"{""description"":""Alright! This one is the littl...",ffe774cc,39,3121,32030,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
1156410,6c930e6e,46ff9d3ad2be09f2,2019-09-28 21:20:41.493000+00:00,"{""duration"":20008,""misses"":0,""event_count"":40,...",ffe774cc,40,2030,32584,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
1156411,a5be6304,46ff9d3ad2be09f2,2019-09-28 21:20:45.499000+00:00,"{""session_duration"":36607,""exit_type"":""game_co...",ffe774cc,41,2010,36607,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
1156412,27253bdc,96d7dc31e822cedc,2019-09-28 21:21:05.670000+00:00,"{""event_code"": 2000, ""event_count"": 1}",ffe774cc,1,2000,0,Tree Top City - Level 3,Clip,TREETOPCITY


In [84]:
(test['timestamp'].max() - test['timestamp'].min()).seconds

75369

In [7]:
train_df = pd.read_csv('../data/preprocessed_train.csv')
test_df = pd.read_csv('../data/preprocessed_test.csv')

In [44]:
assessments = ['Cart Balancer (Assessment)',
                   'Cauldron Filler (Assessment)',
                   'Chest Sorter (Assessment)',
                   'Mushroom Sorter (Assessment)',
                   'Bird Measurer (Assessment)']

In [51]:
test_df[['title_' + title +'_count' for title in assessments]].mean()

title_Cart Balancer (Assessment)_count      18.978
title_Cauldron Filler (Assessment)_count    23.211
title_Chest Sorter (Assessment)_count       17.767
title_Mushroom Sorter (Assessment)_count    21.718
title_Bird Measurer (Assessment)_count      19.953
dtype: float64

In [41]:
train_df['duration_mean'].mean()

174.582954571508

In [12]:
train_df['title'].value_counts(normalize=True)

8     0.234652
9     0.229226
30    0.212380
10    0.168513
4     0.155229
Name: title, dtype: float64

In [21]:
train_df.columns.tolist()

['0_group_count',
 '1_group_count',
 '2_group_count',
 '3_group_count',
 'Activity_count',
 'Assessment_count',
 'Clip_count',
 'Game_count',
 'acc_Bird Measurer (Assessment)',
 'acc_Cart Balancer (Assessment)',
 'acc_Cauldron Filler (Assessment)',
 'acc_Chest Sorter (Assessment)',
 'acc_Mushroom Sorter (Assessment)',
 'accumulated_accuracy',
 'accumulated_accuracy_group',
 'accumulated_actions',
 'accumulated_correct_attempts',
 'accumulated_uncorrect_attempts',
 'accuracy',
 'accuracy_group',
 'duration_mean',
 'event_code_2000_count',
 'event_code_2010_count',
 'event_code_2020_count',
 'event_code_2025_count',
 'event_code_2030_count',
 'event_code_2035_count',
 'event_code_2040_count',
 'event_code_2050_count',
 'event_code_2060_count',
 'event_code_2070_count',
 'event_code_2075_count',
 'event_code_2080_count',
 'event_code_2081_count',
 'event_code_2083_count',
 'event_code_3010_count',
 'event_code_3020_count',
 'event_code_3021_count',
 'event_code_3110_count',
 'event_code_3

In [4]:
count_cols = [col for col in train_df.columns if 'count' in col]

In [6]:
for col in count_cols:
    train_df[col] = train_df[col] / train_df['accumulated_actions']

In [3]:
train_df['accuracy_group'].value_counts(normalize=True)

3    0.500000
0    0.239062
1    0.136292
2    0.124647
Name: accuracy_group, dtype: float64

In [34]:
train_df.groupby('installation_id')['accumulated_actions'].sum().sort_values()

installation_id
08062502          0
9d570bf2          0
db2ab253          0
9b7d6baa          0
ad9d07d6          0
             ...   
96979008     745447
063e3e02     763717
08987c08    1026275
9f399a3a    1040058
f1c21eda    3560604
Name: accumulated_actions, Length: 3614, dtype: int64

In [45]:
train_df.groupby('installation_id')['0_group_count'].count().sort_values(ascending=False).head(10)

installation_id
08987c08    156
f1c21eda    129
9f399a3a     78
8e44d2de     72
123a40a1     64
96979008     64
416398f8     49
2ffb27f9     48
2d6d1ecd     47
063e3e02     46
Name: 0_group_count, dtype: int64

In [53]:
(train_df.groupby('installation_id')['0_group_count'].count().sort_values(ascending=True).head(10000) == 1).sum()

1027

In [38]:
train_df[train_df['installation_id'] != 'f1c21eda']

,0_group_count,1_group_count,2_group_count,3_group_count,Activity_count,Assessment_count,Clip_count,Game_count,acc_Bird Measurer (Assessment),acc_Cart Balancer (Assessment),...,title_event_code_Watering Hole (Activity)_3110_count,title_event_code_Watering Hole (Activity)_4020_count,title_event_code_Watering Hole (Activity)_4021_count,title_event_code_Watering Hole (Activity)_4025_count,title_event_code_Watering Hole (Activity)_4070_count,title_event_code_Watering Hole (Activity)_4090_count,title_event_code_Watering Hole (Activity)_5000_count,title_event_code_Watering Hole (Activity)_5010_count,title_event_code_Welcome to Lost Lagoon!_2000_count,world
0,0,0,0,0,3,0,6,2,-1.0,-1.0,...,0,0,0,0,0,0,0,0,2,3
1,0,0,0,1,4,1,8,4,-1.0,-1.0,...,0,0,0,0,0,0,0,0,2,3
2,1,0,0,1,4,2,8,4,0.0,-1.0,...,0,0,0,0,0,0,0,0,2,3
3,2,0,0,2,9,3,15,7,0.0,-1.0,...,5,9,8,9,4,0,5,5,3,3
4,2,0,1,2,10,3,17,9,0.0,-1.0,...,5,9,8,9,4,0,5,5,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17685,0,1,0,4,7,4,13,10,1.0,1.0,...,4,4,4,4,6,0,3,2,1,0
17686,0,0,0,0,2,0,2,2,-1.0,-1.0,...,0,0,0,0,0,0,0,0,1,1
17687,0,0,0,0,3,0,4,2,-1.0,-1.0,...,0,0,0,0,0,0,0,0,2,1
17688,0,1,0,0,3,1,7,4,-1.0,-1.0,...,0,0,0,0,0,0,0,0,3,3
